## 1.6 Exam vB, PROBLEM 1  
Maximum Points = 14

A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city.  
The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals.  
The locations are divided into three regions: **downtown**, **the suburbs**, and **the countryside**,  
however there is always the possibility the truck breaks down and it goes to the **workshop**.

The following table shows the probabilities of a truck transitioning between these regions at each time step:

Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside | Probability of transitioning to the Workshop
---|---|---|---|---
Downtown | 0.3 | 0.7 | 0 | 0
Suburbs | 0.2 | 0.5 | 0.3 | 0
Countryside | 0 | 0 | 0.5 | 0.5
Workshop | 0 | 0 | 0 | 1

1. If a truck is currently in the downtown, what is the probability that it will be in the countryside region after 10 time steps? **[2p]**

2. If a truck is currently in the downtown, what is the probability that it will be in the countryside region the first time after three time steps or more? **[2p]**

3. Is this Markov chain irreducible? Explain your answer. **[3p]**

4. What is the stationary distribution? Furthermore is it reversible? (Explain your answer) **[3p]**

5. **Advanced question:** What is the expected number of steps it takes starting from the Downtown region to first reach the Workshop?

   *Hint:* to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 50.  
   Motivate your answer in detail. **[4p]**

   You could also solve this question by simulation, but this gives you a maximum of **[2p]**.


In [30]:
# Part 1
import numpy as np
P = np.array([[0.3,0.7,0,0], [0.2,0.5,0.3,0], [0,0,0.5,0.5], [0,0,0,1]])
print("Matrix P is: \n", P)

# After 10 time steps, we simply take the matrix to the power of 10
P10 = np.linalg.matrix_power(P, 10)
print("This is matrix after 10 time stpeps: \n", P10)

# This is the probability were looking at, From Downtown -> Countryside 
print("This is corrcet probability: ", P10[0][2].round(4))
problem1_p1 = P10[0][2]

Matrix P is: 
 [[0.3 0.7 0.  0. ]
 [0.2 0.5 0.3 0. ]
 [0.  0.  0.5 0.5]
 [0.  0.  0.  1. ]]
This is matrix after 10 time stpeps: 
 [[3.39352311e-02 8.26829080e-02 8.48735349e-02 7.98508326e-01]
 [2.36236880e-02 5.75589191e-02 5.96851134e-02 8.59132280e-01]
 [0.00000000e+00 0.00000000e+00 9.76562500e-04 9.99023438e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]
This is corrcet probability:  0.0849


In [31]:
# Part 2
# Fill in the answer to part 2 below

problem1_p2 = 0.79


### 1.7 Part 3
Double click this cell to enter edit mode and write your answer for part 3 below this line.


In [32]:
# Part 3
# Fill in the answer to part 3 below as a boolean
problem1_irreducible = False


In [33]:
# Part 4
# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!
problem1_stationary = np.array([0,0,0,1])
problem1_reversible = True


## Part 4
Double click this cell and write your motivation below this line


### 2.1 Part 5
Double click this cell to enter edit mode and write your answer for part 5 below this line.


In [34]:
# Part 5
# Fill in the answer to part 5 below
# That is, the expected number of steps



import numpy as np

def expected_hitting_time(P, target_states, start_state=None, start_dist=None):
    """
    Compute expected hitting times to a given set of target states in a finite Markov chain.

    Parameters
    ----------
    P : np.ndarray, shape (n, n)
        Transition matrix of the Markov chain.
    target_states : iterable of int
        Indices of the target states.
    start_state : int, optional
        If provided, return the expected hitting time starting from this state.
    start_dist : array-like, shape (n,), optional
        If provided, return the expected hitting time under this initial distribution.

    Returns
    -------
    float or np.ndarray
        - If start_state is given: expected hitting time from that state.
        - If start_dist is given: expected hitting time under that distribution.
        - If neither is given: array h of length n with expected hitting times
          from all states (targets have value 0).

    Notes
    -----
    This solves the linear system

        (I - Q) h_S = 1

    where Q is the submatrix of P restricted to non-target states,
    and 1 is a vector of ones. Assumes that the target set is hit
    with probability 1 from the relevant starting states.
    """
    P = np.asarray(P, dtype=float)
    n = P.shape[0]

    target_states = np.array(sorted(set(target_states)), dtype=int)
    all_states = np.arange(n, dtype=int)

    # Non-target states S
    non_target_states = np.array([s for s in all_states if s not in target_states], dtype=int)

    # If all states are targets, hitting time is identically zero
    if non_target_states.size == 0:
        h = np.zeros(n, dtype=float)
        if start_state is not None:
            return float(h[start_state])
        if start_dist is not None:
            start_dist = np.asarray(start_dist, dtype=float)
            return float(start_dist @ h)
        return h

    # Build Q and solve (I - Q) h_S = 1
    Q = P[np.ix_(non_target_states, non_target_states)]
    I = np.eye(Q.shape[0])
    ones = np.ones(Q.shape[0])

    # Solve for h_S
    h_S = np.linalg.solve(I - Q, ones)

    # Put back into full vector h of length n
    h = np.zeros(n, dtype=float)
    h[target_states] = 0.0
    for idx, s in enumerate(non_target_states):
        h[s] = h_S[idx]

    # Return according to user request
    if (start_state is not None) and (start_dist is not None):
        raise ValueError("Provide either start_state or start_dist, not both.")

    if start_state is not None:
        return float(h[start_state])

    if start_dist is not None:
        start_dist = np.asarray(start_dist, dtype=float)
        if start_dist.shape[0] != n:
            raise ValueError("start_dist must have length equal to number of states.")
        return float(start_dist @ h)

    return h


ET_suburbs_to_downtown = expected_hitting_time(P, target_states=[3], start_state=0)
print(ET_suburbs_to_downtown)  # ~3.846153846153846 (50/13)

problem1_ET = ET_suburbs_to_downtown


7.7142857142857135


---------

## 2.2 Exam vB, PROBLEM 2  
Maximum Points = 13  

You are given a “Data Science Salaries” dataset found in `data/salaries.csv`, which contains  
employment information of data scientists up to 2023 and the salary obtained.  
Your task is to train a linear regression model to predict the salary of a data scientist  
based on the employment information.

To evaluate your model, you will split the dataset into a training set and a testing set.  
You will use the training set to train your model, and the testing set to evaluate its performance.

Experience level:  
- 0 = Entry Level  
- 1 = Mid Level  
- 2 = Senior Level  
- 3 = Executive Level  

Employment type:  
- 0 = Part Time  
- 1 = Full Time  
- 2 = Contractor  
- 3 = Freelancer  

1. Load the data into a pandas dataframe `problem2_df`.  
   Based on the column names, figure out what are the features and the target and fill in the answer in the correct cell below. **[1p]**

2. Split the data into train and test. **[1p]**

3. Train the model. **[1p]**

4. Come up with a reasonable metric and compute it.  
   Provide plots that show the performance of the model.  
   Reason about the performance. **[4p]**

5. Predict the 2023 salary of a data scientist that works full time (1) at mid employment level (1) with 0 remote ratio.  
   Then, looking at the output of `problem2_model.coef_`, which are the coefficients of the linear model,  
   would a higher remote ratio result in a higher predicted salary or vice versa? **[3p]**

6. **Advanced question:** On the test set, plot the empirical distribution function of the residual  
   with confidence bands (i.e. using the DKW inequality and 95% confidence).  
   What does the confidence band tell us?  
   What can the confidence band be used for? **[3p]**


In [35]:
# Part 1
# Let problem2_df be the pandas dataframe that contains the data from the file
# data/abalone.csv
import pandas as pd
data = pd.read_csv("../../../data/salaries.csv")
print(data.columns)
print(data.head(5))
problem2_df = data


Index(['work_year', 'experience_level', 'employment_type', 'salary_in_usd',
       'remote_ratio'],
      dtype='object')
   work_year  experience_level  employment_type  salary_in_usd  remote_ratio
0       2023                 2                1          85847           100
1       2023                 1                2          30000           100
2       2023                 1                2          25500           100
3       2023                 2                1         175000           100
4       2023                 2                1         120000           100


In [36]:
# Part 1
# Fill in the features as a list of strings of the names of the columns
problem2_features = ["work_year", "experience_level", "employment_type", "remote_ratio"]
# Fill in the target as a string with the correct column name
problem2_target = "salary_in_usd"


In [37]:
# Part 2
# Split the data into train and test using train_test_split
# keep the train size as 0.8 and use random_state=42
from sklearn.model_selection import train_test_split

X = problem2_df[problem2_features]
y = problem2_df[problem2_target]

problem2_X_train,problem2_X_test,problem2_y_train,problem2_y_test = train_test_split(X, y, train_size=0.8, random_state=42)


In [38]:
# Part 3
# Include the necessary imports
# Initialize your linear regression model
from sklearn.linear_model import LinearRegression

problem2_model = LinearRegression()
# Train your model on the training data
problem2_model.fit(problem2_X_train,problem2_y_train)
print("Coefficients: ", problem2_model.coef_)


Coefficients:  [ 1.32911910e+04  3.89237923e+04 -3.47903893e+03 -1.30294752e+01]


### 2.3 Part 4  
Double click this cell to enter edit mode and write your answer for part 4 below this line.


In [39]:
# Part 4
# Write the code to diagnose your model
from sklearn.metrics import mean_squared_error 

y_train_pred = problem2_model.predict(problem2_X_train)
train_mse = mean_squared_error(problem2_y_train, y_train_pred)
#train_rmse = np.sqrt(train_mse)
print(train_mse)


3103548309.9614615


### 2.4 Part 5  
Double click this cell to enter edit mode and write your answer for part 5 below this line.


5. Predict the 2023 salary of a data scientist that works full time (1) at mid employment level (1) with 0 remote ratio.  
   Then, looking at the output of `problem2_model.coef_`, which are the coefficients of the linear model,  
   would a higher remote ratio result in a higher predicted salary or vice versa? **[3p]**



In [44]:
# Part 5
# Put the code for part 5 below this line
# Part 5
new_data = pd.DataFrame({
    "work_year": [2023],
    "experience_level": [1],   # Mid level
    "employment_type": [1],    # Full time
    "remote_ratio": [0]        # No remote work
})
# Use the trained model to predict the salary
problem2_predicted_salary = problem2_model.predict(new_data)[0]

print("Predicted salary:", problem2_predicted_salary)

# Check the coefficient for remote_ratio to understand its influence
remote_ratio_coef = problem2_model.coef_[problem2_features.index("remote_ratio")]   
print("Coefficient for remote_ratio:", remote_ratio_coef)

# Interpretation comment:
# If remote_ratio_coef > 0 => higher remote_ratio gives higher predicted salary.
# If remote_ratio_coef < 0 => higher remote_ratio gives lower predicted salary.



Predicted salary: 121940.38437091932
Coefficient for remote_ratio: -13.029475233542966


### 2.5 Part 6  
Double click this cell to enter edit mode and write your answer for part 6 below this line.


6. **Advanced question:** On the test set, plot the empirical distribution function of the residual  
   with confidence bands (i.e. using the DKW inequality and 95% confidence).  
   What does the confidence band tell us?  
   What can the confidence band be used for? **[3p]**

In [ ]:
# Part 6
# Put the code for part 6 below this line



## 2.6 Exam vB, PROBLEM 3  
Maximum Points = 13  

### 2.7 Random variable generation

1. **[4p]** Using inversion sampling, construct 1000 samples from the below distribution  

$$
F[x] =
\begin{cases}
0, & x \le 0 \\
e^x - 1, & 0 < x < \ln(2) \\
1, & x \ge \ln(2)
\end{cases}
$$

2. **[2p]** Use the above 1000 samples to estimate the mean and variance.

3. **[4p]** Using the Accept–Reject sampler (Algorithm 1 in TFDS notes) construct 1000 samples  
   from the same distribution.  
   What proposal distribution did you choose and why?  
   What proportion of samples were accepted?

4. **[3p]** Explain if it is possible to sample from the density  

$$
f(x) = C e^{-(x^2 - 2)^2}
$$

using the Accept–Reject sampler (Algorithm 1 in TFDS notes) with sampling density  
given by the Gaussian.  
Here \( C \) is a constant to make sure that \( f \) is a density, and it is between roughly  
1.34 and 1.35.


In [48]:
# Part 1
# Write your code below
# Part 1
# Put the resulting samples into the following variable
n_samples = 1000
samples = []

while len(samples) < n_samples:
    U = np.random.uniform(0.0, 1.0)
    x = np.log(U + 1)
    
    samples.append(x)
    
print("Length of samples: ", len(samples))

# Alternative for the loop function:
# n_samples = 1000
# U = np.random.uniform(0, 1, n_samples)
# samples = np.log(1 + U)

problem3_samples = samples


Length of samples:  1000


In [49]:
# Part 2
problem3_mean = np.mean(problem3_samples)
problem3_variance = np.var(problem3_samples)

print("This is the mean: ", problem3_mean)
print("This is the variance: ", problem3_variance)


This is the mean:  0.3962284198727792
This is the variance:  0.03843691429340791


In [52]:
# Part 3
# Write your code to solve the problem below

# Part 3
# Write your answer in this cell below

samples = []
n_samples = 1000
n_proposed = 0

while len(samples) < n_samples:
    x = np.random.uniform(0, np.log(2))
    
    u = np.random.uniform(0, 1)
    
    acceptance_rate = np.exp(x) / 2.0
    
    if (u <= acceptance_rate):
        samples.append(x)
        
    n_proposed += 1
    
    


problem3_samples_accept_reject = np.array(samples)

# Put your answer for the proportion of samples accepted below
problem3_acceptance_rate = len(samples) / n_proposed

print("Acceptance Rate: ", problem3_acceptance_rate)

Acceptance Rate:  0.7288629737609329


In [65]:
import numpy as np

def f_x(x):
    return np.exp(x)          # target pdf

def g_x(x):
    return 1/np.log(2)        # uniform(0, ln 2) pdf

xs = np.linspace(0, np.log(2), 1000)
ratio = f_x(xs) / g_x(xs)
M_num = ratio.max()

print("Answer: ", 2*np.log(2))

print("Numeric M ≈", M_num)   # should be close to 2*np.log(2)


Answer:  1.3862943611198906
Numeric M ≈ 1.3862943611198906


4. **[3p]** Explain if it is possible to sample from the density  

$$
f(x) = C e^{-(x^2 - 2)^2}
$$

using the Accept–Reject sampler (Algorithm 1 in TFDS notes) with sampling density  
given by the Gaussian.  
Here \( C \) is a constant to make sure that \( f \) is a density, and it is between roughly  
1.34 and 1.35.

In [74]:

def f_tilde(x):
    return (np.exp(-(x**2 - 2)**2))


# Because when nothing is stated, the Gaussian used as a proposal distribution is always assumed to be the 
# standard normal N(0,1)

def g_x(x): 
    return (1/np.sqrt(2*np.pi)) * np.exp(-x**2/2)

n_samples = 1000
samples = []
n_proposed = 0

xs = np.linspace(-5, 5, 10000)
ratio = f_tilde(xs) / g_x(xs)
M_est = ratio.max()
print("M est is: ", M_est)


while (len(samples) < n_samples):
    x = np.random.randn()
    
    u = np.random.uniform(0, 1)
    
    if u <= f_tilde(x) / (M_est * g_x(x)):
        samples.append(x)
    
    n_proposed += 1


problem4_samples_accept_reject = np.array(samples)
print("Samples total: ", len(problem4_samples_accept_reject))

# Put your answer for the proportion of samples accepted below
problem4_acceptance_rate = len(samples) / n_proposed

print("Acceptance Rate: ", problem4_acceptance_rate)

M est is:  7.253161412670422
Samples total:  1000
Acceptance Rate:  0.18789928598271327
